In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import os
import gmaps
#import gmplot
#from geolocation.main import GoogleMaps
#from geolocation.distance_matrix.client import DistanceMatrixApiClient
from yelpapi import YelpAPI
from pprint import pprint
from config import apikey
from config import gkey
import csv
from yelp.client import Client

In [2]:
# Get yo' Key
yelp_api = YelpAPI(apikey)
#print(yelp_api)

In [3]:
# Print each city's input into different lists. 
# We also need information on the restaurant's name, rating, location, lat/long and category
# To do this we're going to set up a list of cities, and a dictionary to hold it all in.
yelp_api = YelpAPI(apikey)
city_names = ["Oceanside", "Encinitas", "Carlsbad"]
data = {
    city: {
        'names': [],
        'ratings': [],
        'categories': [],
        'location':[],
        'lat':[],
        'lng':[]
    }
    for city in city_names
}
# Then we're gonna print just to see that our lists and dictionaries are set up correctly
print(json.dumps(data, indent=4))

#Now we just run a loop through the city names to append the data into the city dictionaries where they belong
for city in city_names:
    search_results = yelp_api.search_query(location=city, 
                                           term="restaurant", 
                                           limit = 50) 
                                           #offset=100)
    for b in search_results['businesses']:
        try:
            data[city]['names'].append(b["name"])
            data[city]['ratings'].append(b["rating"])
            data[city]['categories'].append(b["categories"][0]['title']+"/"+b["categories"][0]["alias"])
            data[city]['location'].append(b['location']['city'])
            data[city]['lat'].append(b['coordinates']['latitude'])
            data[city]['lng'].append(b['coordinates']['longitude'])
        except:
            data[city]['names'].append(np.nan)
            data[city]['ratings'].append(np.nan)
            data[city]['categories'].append(np.nan)
            data[city]['location'].append(np.nan)
            data[city]['lat'].append(np.nan)
            data[city]['lng'].append(np.nan)

# You can run a test here for length checks.

# print(len(search_results['businesses']))
# print(len(name))
# print(len(rating))
# print(len(category))
# print(len(city_list))

# Then we pretty print the dictionaries to make sure they're all populated
pprint(data, indent=4)

{
    "Oceanside": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    },
    "Encinitas": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    },
    "Carlsbad": {
        "names": [],
        "ratings": [],
        "categories": [],
        "location": [],
        "lat": [],
        "lng": []
    }
}
{   'Carlsbad': {   'categories': [   'American (New)/newamerican',
                                      'Cocktail Bars/cocktailbars',
                                      'Seafood/seafood',
                                      'Breakfast & Brunch/breakfast_brunch',
                                      'American (New)/newamerican',
                                      'American (New)/newamerican',
                                      'Sandwiches/sandwiches',
                                      'Pubs/pubs',
       

In [4]:
# Now that we have everything in seperate dictionaries, let's put it together into 3 seperate dataframes
Oceanside_df = pd.DataFrame.from_dict(data["Oceanside"])
Carlsbad_df = pd.DataFrame.from_dict(data["Carlsbad"])
Encinitas_df = pd.DataFrame.from_dict(data["Encinitas"])

# We'll print the numbers to make sure they're at 50 each
print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

# Then we're dropping any null values
Oceanside_df = Oceanside_df.dropna(how='any')
Carlsbad_df = Carlsbad_df.dropna(how='any')
Encinitas_df = Encinitas_df.dropna(how='any')

# And printing again to make sure those are dropped if there are any
print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

50
50
50
50
50
50


In [5]:
# Since some of our data isn't from the cities we're interested in, let's fix that.

# First we'll set the index to "location"
Oceanside_df = Oceanside_df.set_index("location")
Carlsbad_df = Carlsbad_df.set_index("location")
Encinitas_df= Encinitas_df.set_index("location")

# Then we're going to run a loc on the cities we want
# This will make sure we only have the cities we are interested in
Oceanside_df=Oceanside_df.loc["Oceanside",:]
Carlsbad_df=Carlsbad_df.loc["Carlsbad",:]
Encinitas_df=Encinitas_df.loc["Encinitas",:]

# We'll print the lengths to see what's left, and the head to make sure the DFs are what we want

print(len(Oceanside_df))
print(len(Carlsbad_df))
print(len(Encinitas_df))

Oceanside_df.head(20)

46
45
44


,names,ratings,categories,lat,lng
location,,,,,
Oceanside,Flying Pig Pub & Kitchen,4.5,Pubs/pubs,33.188440,-117.374070
Oceanside,Miss Kim's by That Boy Good,4.5,Barbeque/bbq,33.196043,-117.380008
Oceanside,Local Tap House & Kitchen,4.5,American (New)/newamerican,33.192741,-117.376506
Oceanside,Stratford at the Harbor,4.5,American (Traditional)/tradamerican,33.205040,-117.389237
Oceanside,The Miller's Table,5.0,Wine Bars/wine_bars,33.190360,-117.374490
Oceanside,Beach Break Cafe,4.5,Breakfast & Brunch/breakfast_brunch,33.175083,-117.361902
Oceanside,Miramar Fish Tacos & Beer,4.5,Seafood/seafood,33.226692,-117.326496
Oceanside,Rockin' Baja Lobster,4.0,Mexican/mexican,33.204210,-117.390270
Oceanside,Felix's BBQ With Soul,4.5,Southern/southern,33.210161,-117.311183


In [6]:
# Let's see who our top 5 competitors are in each city
# We'll start with Oceanside
SortedOceanside=Oceanside_df.sort_values("ratings", ascending=False)
SortedCarlsbad=Carlsbad_df.sort_values("ratings",ascending=False)
SortedEncinitas=Encinitas_df.sort_values("ratings",ascending=False)

Top5Oceanside = SortedOceanside.head(5).reset_index()
Top5Oceanside

,location,names,ratings,categories,lat,lng
0,Oceanside,Elena's Cafe French Crepes,5.0,Creperies/creperies,33.196777,-117.380023
1,Oceanside,The Miller's Table,5.0,Wine Bars/wine_bars,33.190360,-117.374490
2,Oceanside,Come On Over Cafe,5.0,Cafes/cafes,33.199474,-117.345764
3,Oceanside,O'side Sports Bar & Grill,5.0,Sports Bars/sportsbars,33.194526,-117.378585
4,Oceanside,Board & Brew - Oceanside,5.0,Beer Bar/beerbar,33.186110,-117.328900


In [7]:
Top5Carlsbad = SortedCarlsbad.head(5).reset_index()
Top5Carlsbad

,location,names,ratings,categories,lat,lng
0,Carlsbad,2051 Cafe,5.0,Cafes/cafes,33.122497,-117.276417
1,Carlsbad,Harumama Carlsbad,5.0,Ramen/ramen,33.161560,-117.346860
2,Carlsbad,Bigfoot Natural Cafe,5.0,Vegan/vegan,33.075410,-117.237240
3,Carlsbad,Campfire,4.5,American (New)/newamerican,33.161990,-117.351050
4,Carlsbad,Paon Restaurant and Wine Bar,4.5,Wine Bars/wine_bars,33.160417,-117.348055


In [8]:
Top5Encinitas = SortedEncinitas.head(5).reset_index()
Top5Encinitas

,location,names,ratings,categories,lat,lng
0,Encinitas,Chiko,5.0,Asian Fusion/asianfusion,33.05006,-117.294160
1,Encinitas,Plum Encinitas,5.0,Thai/thai,33.04551,-117.293450
2,Encinitas,Chiosco Caffè,5.0,Cafes/cafes,33.05391,-117.261000
3,Encinitas,Buona Forchetta,5.0,Pizza/pizza,33.05359,-117.296590
4,Encinitas,Rooster Ramen,4.5,Ramen/ramen,33.06449,-117.302569


In [9]:
Oceanside_df = Oceanside_df.reset_index()
GroupedOceanside = Oceanside_df.groupby("categories").count()
GroupedOceanside
#GroupedOceanside.head()

,location,names,ratings,lat,lng
categories,,,,,
American (New)/newamerican,5,5,5,5,5
American (Traditional)/tradamerican,2,2,2,2,2
Barbeque/bbq,2,2,2,2,2
Beer Bar/beerbar,1,1,1,1,1
Breakfast & Brunch/breakfast_brunch,3,3,3,3,3
Breweries/breweries,1,1,1,1,1
Burgers/burgers,1,1,1,1,1
Cafes/cafes,1,1,1,1,1
Caribbean/caribbean,1,1,1,1,1


In [10]:
#for data in Oceanside_df:
#    hashStr = hash(Oceanside_df["categories"])
#    r = (hashStr & 0xFF0000) >> 16;
#    g = (hashStr & 0x00FF00) >> 8;
#    b = hashStr & 0x0000FF;

#print(r,g,b)

In [11]:
# Now we do heatmaps. First we're loading up the figures we want
gmaps.configure(api_key=gkey)
Carlsbad_coordinates = (33.1269333,-117.339839)
Carlsbad_fig=gmaps.figure(center=Carlsbad_coordinates, zoom_level=12)
Carlsbad_fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
Oceanside_coordinates = (33.1959, -117.3795)
Oceanside_fig = gmaps.figure(center=Oceanside_coordinates, zoom_level=12)
Oceanside_fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
Encinitas_coordinates = (33.0450, -117.2920)
Encinitas_fig = gmaps.figure(center=Encinitas_coordinates, zoom_level=14)
Encinitas_fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
Oceanside_locations = Oceanside_df[["lat","lng"]].astype(float)
# create empty lists for colors and category
# store the category and the color in their respective lists
# turn lists into a legend


for i, row in Oceanside_df.iterrows():
    print(row['categories'])
    Olat = row['lat']
    Olng = row['lng']
    hashStr = hash(row['categories'])
    #Friend helped me with the bitshift shenanigans
    # Hash turns a string into an int, it puts numbers based on what the letters are
    # We're then going to manipulate the hashStr to get our RGB values
    # & is a bit mask, whatever numbers match up with the FFs are kept, but in binary code. 
    # >> is a bitshift, where we're taking the binary, and we're moving them 16 zeroes over to the left 
    #r = (hashStr & 0xFF0000) >> 16;
    #g = (hashStr & 0x00FF00) >> 8;
    #b = hashStr & 0x0000FF;
    Oceanside_fig.add_layer(gmaps.symbol_layer([(Olat, Olng)], stroke_color =('Blue'),stroke_opacity=1.0, fill_color=('Blue'), scale =3))
    #Oceanside_fig.add_layer(gmaps.symbol_layer(Oceanside_locations, fill_color="({}, {}, {})".format(r, g,b), stroke_color = 'rgba(0,0,0,0)', scale =.05))
Oceanside_fig
    

Pubs/pubs
Barbeque/bbq
American (New)/newamerican
American (Traditional)/tradamerican
Wine Bars/wine_bars
Breakfast & Brunch/breakfast_brunch
Seafood/seafood
Mexican/mexican
Southern/southern
Burgers/burgers
Peruvian/peruvian
American (New)/newamerican
Ramen/ramen
Breakfast & Brunch/breakfast_brunch
Ramen/ramen
Japanese/japanese
American (New)/newamerican
American (New)/newamerican
Steakhouses/steak
Caribbean/caribbean
Gastropubs/gastropubs
American (New)/newamerican
Breweries/breweries
Italian/italian
Cheesesteaks/cheesesteaks
Barbeque/bbq
Fish & Chips/fishnchips
Seafood/seafood
Seafood/seafood
Beer Bar/beerbar
Breakfast & Brunch/breakfast_brunch
Italian/italian
Seafood/seafood
Thai/thai
Indonesian/indonesian
Poke/poke
Mexican/mexican
Sports Bars/sportsbars
Cafes/cafes
Vietnamese/vietnamese
American (Traditional)/tradamerican
Vietnamese/vietnamese
Vietnamese/vietnamese
Mexican/mexican
Creperies/creperies
Hawaiian/hawaiian


Figure(layout=FigureLayout(height='420px'))

In [15]:
#Oceanside_df = Oceanside_df.set_index("categories")
print(Oceanside_df["categories"])

0                               Pubs/pubs
1                            Barbeque/bbq
2              American (New)/newamerican
3     American (Traditional)/tradamerican
4                     Wine Bars/wine_bars
5     Breakfast & Brunch/breakfast_brunch
6                         Seafood/seafood
7                         Mexican/mexican
8                       Southern/southern
9                         Burgers/burgers
10                      Peruvian/peruvian
11             American (New)/newamerican
12                            Ramen/ramen
13    Breakfast & Brunch/breakfast_brunch
14                            Ramen/ramen
15                      Japanese/japanese
16             American (New)/newamerican
17             American (New)/newamerican
18                      Steakhouses/steak
19                    Caribbean/caribbean
20                  Gastropubs/gastropubs
21             American (New)/newamerican
22                    Breweries/breweries
23                        Italian/

In [16]:
# Let's break the dataframes into their categories
# We're going to have to get a list of each 

In [17]:
# Next is Encinitas' heatmap
# encinitas = green

Encinitas_locations = Encinitas_df[["lat","lng"]].astype(float)
for i, row in Encinitas_df.iterrows():
    print(row['categories'])
    Elat = row['lat']
    Elng = row['lng']
    #hashStr = hash(row['categories'])
    Encinitas_fig.add_layer(gmaps.symbol_layer([(Elat, Elng)], stroke_color =('Green'),stroke_opacity=1.0, fill_color=('Green'), scale =3))
Encinitas_fig
# carlsbad = dark orange

American (Traditional)/tradamerican
American (New)/newamerican
Latin American/latin
Breakfast & Brunch/breakfast_brunch
Seafood/seafood
Filipino/filipino
Gastropubs/gastropubs
Tacos/tacos
Mediterranean/mediterranean
Sandwiches/sandwiches
American (New)/newamerican
American (New)/newamerican
Sushi Bars/sushi
International Grocery/intlgrocery
Himalayan/Nepalese/himalayan
Desserts/desserts
Italian/italian
Seafood/seafood
Cafes/cafes
Seafood/seafood
Italian/italian
Thai/thai
Hawaiian/hawaiian
Thai/thai
Wine Bars/wine_bars
Cocktail Bars/cocktailbars
Vietnamese/vietnamese
American (New)/newamerican
Diners/diners
American (New)/newamerican
Vietnamese/vietnamese
Sandwiches/sandwiches
Beer Bar/beerbar
Sandwiches/sandwiches
Mediterranean/mediterranean
Breakfast & Brunch/breakfast_brunch
Fast Food/hotdogs
Pizza/pizza
Asian Fusion/asianfusion
Pubs/pubs
Gluten-Free/gluten_free
Brazilian/brazilian
Asian Fusion/asianfusion
Ramen/ramen


Figure(layout=FigureLayout(height='420px'))

In [18]:
Carlsbad_locations = Carlsbad_df[["lat","lng"]].astype(float)
for i, row in Carlsbad_df.iterrows():
    print(row['categories'])
    Clat = row['lat']
    Clng = row['lng']
    #hashStr = hash(row['categories'])
    Carlsbad_fig.add_layer(gmaps.symbol_layer([(Clat, Clng)], stroke_color =('rgba(250, 73, 35, 1)'),stroke_opacity=1.0, fill_color=('rgba(250, 73, 35, 1)'), scale =3))
Carlsbad_fig

American (New)/newamerican
Cocktail Bars/cocktailbars
Seafood/seafood
Breakfast & Brunch/breakfast_brunch
American (New)/newamerican
American (New)/newamerican
Sandwiches/sandwiches
Pubs/pubs
Gastropubs/gastropubs
Comfort Food/comfortfood
Coffee & Tea/coffee
Seafood/seafood
Wine Bars/wine_bars
American (New)/newamerican
American (New)/newamerican
Meat Shops/meats
Cafes/cafes
Peruvian/peruvian
Steakhouses/steak
Vegan/vegan
Mexican/mexican
Burgers/burgers
Japanese/japanese
Sushi Bars/sushi
Mexican/mexican
Italian/italian
American (New)/newamerican
Delis/delis
Cocktail Bars/cocktailbars
Coffee & Tea/coffee
Ramen/ramen
Italian/italian
Seafood/seafood
Venues & Event Spaces/venues
Italian/italian
Ramen/ramen
Gluten-Free/gluten_free
Burgers/burgers
Sushi Bars/sushi
Pizza/pizza
Indian/indpak
Indian/indpak
Wine Bars/wine_bars
Asian Fusion/asianfusion
American (Traditional)/tradamerican


Figure(layout=FigureLayout(height='420px'))

In [19]:
#hashStr = hash("hello world")
#print(hashStr & 0xFF0000)
#print(hashStr & 0x00FF00)
#print(hashStr & 0x0000FF)
#print('-------')

#print((hashStr & 0xFF0000) >> 16)
#print((hashStr & 0x00FF00) >> 8)
#print(hashStr & 0x0000FF)